# Lab 2 — Cleaning & Normalization Pipeline (News Classification)

This notebook should run **top-to-bottom** in Google Colab or locally.

**Outputs**:
- `data/processed_v2/train_cleaned.csv`
- `docs/audit_summary_lab2.md`

In [1]:

from ua_datasets import NewsClassificationDataset
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
import json
import sys
import os

In [ ]:
if 'google.colab' in sys.modules:
    if not os.path.exists('/content/nlp'):
        !git clone https://github.com/jaYulichka46/nlp.git
    
    %cd /content/nlp
    !pip install ftfy regex pandas ua-datasets -q
    sys.path.append('/content/nlp')

    FOLDER_ID = '1vekLcayf46Zvlc3u_Px-PK2MchheqDl1'
    
    os.makedirs('data', exist_ok=True)
    !gdown --folder https://drive.google.com/drive/folders/{FOLDER_ID} -O data/
    
    raw_data_path = 'data/raw'
else:
    sys.path.append(os.path.abspath('..'))
    raw_data_path = '../data/raw'

In [3]:
csv_file = os.path.join(raw_data_path, 'train.csv')

if not os.path.exists(csv_file):
    os.makedirs(raw_data_path, exist_ok=True)
    NewsClassificationDataset(root=raw_data_path)

df = pd.read_csv(csv_file)

print(f"Всього новин: {len(df)}")
display(df[['title', 'text']].head(3))

Всього новин: 120417


,title,text
0,"ВЕРНИДУБ: «Наносимо 25 ударів, 15 у ворота, а ...",Головний тренер солігорського «Шахтаря» Юрій В...
1,"У ""Київстар"" заявляють, що їх обшукала ДФС",Про це на своїй сторінці у Facebook написав пр...
2,В 2016 році 1% найзаможніших людей вперше ста...,Про це повідомляється в доповіді некомерційної...


In [4]:
from src.preprocess import preprocess

processed_results = df['text'].astype(str).apply(preprocess)

df['clean_text'] = processed_results.apply(lambda d: d['clean'])
df['sentences'] = processed_results.apply(lambda d: d.get('sentences', []))

display(df[['text', 'clean_text', 'sentences']].head())

,text,clean_text,sentences
0,Головний тренер солігорського «Шахтаря» Юрій В...,"Головний тренер солігорського ""Шахтаря"" Юрій В...","[Головний тренер солігорського ""Шахтаря"" Юрій ..."
1,Про це на своїй сторінці у Facebook написав пр...,Про це на своїй сторінці у Facebook написав пр...,[Про це на своїй сторінці у Facebook написав п...
2,Про це повідомляється в доповіді некомерційної...,Про це повідомляється в доповіді некомерційної...,[Про це повідомляється в доповіді некомерційно...
3,Легенда НБА Шакіл О’Ніл продав свій маєток у ...,Легенда НБА Шакіл О'Ніл продав свій маєток у Ф...,[Легенда НБА Шакіл О'Ніл продав свій маєток у ...
4,Засновник фінансової піраміди B2B Jewelry Мик...,Засновник фінансової піраміди B2B Jewelry Мико...,[Засновник фінансової піраміди B2B Jewelry Мик...


In [5]:
print("До / Після:")
display(df[['text', 'clean_text']].sample(min(15, len(df)), random_state=42))

До / Після:


,text,clean_text
1592,"Двоє юристів з Австралії вважають, що зовсім с...","Двоє юристів з Австралії вважають, що зовсім с..."
105053,"Про це заявив речник Кремля Дмитро Пєсков, пер...","Про це заявив речник Кремля Дмитро Пєсков, пер..."
53676,Про це під час прес-конференції заявив президе...,Про це під час прес-конференції заявив президе...
76048,Фото та відео незвичайного для травня природно...,Фото та відео незвичайного для травня природно...
7067,Про це йдеться в указі президента. Три частини...,Про це йдеться в указі президента. Три частини...
11141,"У суботу, 10 квітня, на київське «Динамо» чека...","У суботу, 10 квітня, на київське ""Динамо"" чека..."
97072,"Дружина Міхаеля Шумахера, Корінна, розповіла п...","Дружина Міхаеля Шумахера, Корінна, розповіла п..."
11163,Відомий український тренер Вадим Євтушенко под...,Відомий український тренер Вадим Євтушенко под...
65113,Про це повідомив Укргідрометцентр. Очікується ...,Про це повідомив Укргідрометцентр. Очікується ...
1599,Відомий британський боксер Ділліан Вайт поділи...,Відомий британський боксер Ділліан Вайт поділи...


In [6]:
def get_stats(series: pd.Series):
    lens = series.astype(str).str.len()
    return {
        'n': int(len(series)),
        'empty_%': round(float((series.astype(str).str.strip() == '').mean() * 100), 4),
        'very_short_%(<20_chars)': round(float((lens < 20).mean() * 100), 4),
        'min_len': int(lens.min()),
        'median_len': int(lens.median()),
        'max_len': int(lens.max()),
        'duplicate_%': round(float(series.astype(str).duplicated().mean() * 100), 4),
    }

raw_stats = get_stats(df['text'])
clean_stats = get_stats(df['clean_text'])

print("Статистика RAW:")
print(raw_stats)
print("\nСтатистика CLEAN:")
print(clean_stats)

mask_counts = {
    '<URL>': int(df['clean_text'].str.contains('<URL>', regex=False).sum()),
    '<EMAIL>': int(df['clean_text'].str.contains('<EMAIL>', regex=False).sum())
}
print("\nКількість замаскованих PII:", mask_counts)

Статистика RAW:
{'n': 120417, 'empty_%': 0.0, 'very_short_%(<20_chars)': 0.0075, 'min_len': 2, 'median_len': 896, 'max_len': 61263, 'duplicate_%': 0.5838}

Статистика CLEAN:
{'n': 120417, 'empty_%': 0.0, 'very_short_%(<20_chars)': 0.0083, 'min_len': 1, 'median_len': 894, 'max_len': 61251, 'duplicate_%': 0.6934}

Кількість замаскованих PII: {'<URL>': 915, '<EMAIL>': 452}


In [7]:
edge_path = Path('../tests/edge_cases.jsonl')

if edge_path.exists():
    edge_rows = [json.loads(line) for line in edge_path.read_text(encoding='utf-8').splitlines() if line.strip()]
    
    preview = []
    for r in edge_rows:
        out = preprocess(r['raw_text'])
        preview.append({
            'id': r['id'],
            'raw': r['raw_text'],
            'clean': out['clean'],
            'sentences': out.get('sentences', []),
            'expected': r.get('expected_behavior','')
        })
    
    print(f"Edge Cases: {len(preview)}")
    display(pd.DataFrame(preview))
else:
    print(f"Файл {edge_path} не знайдено")

Edge Cases: 20


,id,raw,clean,sentences,expected
0,1,«Шахтаря»,"""Шахтаря""","[""Шахтаря""]","уніфікація лапок-ялинок, лема 'шахтар'"
1,2,у м. Києві,у м. Києві,[у м. Києві],не розбивати речення після скорочення 'м.'
2,3,рахунок 1:2,рахунок 1:2,[рахунок 1:2],збереження формату спортивного рахунку (не час)
3,4,35 %,35 %,[35 %],збереження знаку %
4,5,|курс долара|євро|курс гривні|долар|гривня|Нов...,курс долара євро курс гривні долар гривня Новини,[курс долара євро курс гривні долар гривня Нов...,заміна вертикальних рисочок на пробіли
5,6,"""""Київстар""""","""Київстар""","[""Київстар""]",видалення подвійних лапок (артефактів CSV)
6,7,$76 трлн,$76 трлн,[$76 трлн],збереження валюти та скорочення 'трлн'
7,8,О’Ніл,О'Ніл,[О'Ніл],уніфікація спеціального апострофа
8,9,SpaceX-розробка,SpaceX-розробка,[SpaceX-розробка],збереження дефісу в змішаному (латиниця/кирили...
9,10,B2B Jewelry,B2B Jewelry,[B2B Jewelry],збереження латиниці та цифр у назві бренду


In [8]:
output_csv = Path('../data/processed_v2/processed_v2.csv')
output_csv.parent.mkdir(parents=True, exist_ok=True)
df[['target', 'clean_text', 'sentences']].to_csv(output_csv, index=False, encoding='utf-8')

summary = [
    '# Lab 2 Audit Summary',
    f"Generated: {datetime.now().isoformat(timespec='seconds')}\n",
    '## Raw stats'
]


for k, v in raw_stats.items():
    summary.append(f"- **{k}**: {v}")

summary.extend(['\n## Processed stats'])
for k, v in clean_stats.items():
    summary.append(f"- **{k}**: {v}")

summary.extend(['\n## Masking counts'])
for k, v in mask_counts.items():
    summary.append(f"- **{k}**: {v}")

summary.extend([
    '\n## Notes',
    '- **Покращення:** Успішно уніфіковано новинні лапки («») та апострофи. Виправлено проблеми з кодуванням за допомогою ftfy.',
    '- **Маскування:** Видалено прямі посилання на джерела та контактні email-адреси.',
    '- **Спліттер:** Реалізовано безпечне розбиття на речення, яке не ламається на географічних скороченнях (м., вул.) та ініціалах.',
])

out_path = Path('../docs/audit_summary_lab2.md')
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text('\n'.join(summary), encoding='utf-8')
print(f"Audit Summary: {out_path}")

Audit Summary: ..\docs\audit_summary_lab2.md
